In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings



In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
# user information
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
user_df = pd.read_csv('/gdrive/My Drive/Recom_Engine/user.txt', sep='|', names=u_cols)

In [0]:
user_df.head()


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [0]:
# ratings given by the user
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
rating_df = pd.read_csv('/gdrive/My Drive/Recom_Engine/rating.txt', sep='\t', names=r_cols)

In [0]:
rating_df.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
# movies information

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movie_df = pd.read_csv('/gdrive/My Drive/Recom_Engine/item.txt', sep='|', names=i_cols,encoding='latin-1')

In [0]:
movie_df.head(10)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [0]:
print('user shape' , user_df.shape)
print('rating shape' , rating_df.shape)
print('movie shape' , movie_df.shape)

user shape (943, 5)
rating shape (100000, 4)
movie shape (1682, 24)


In [0]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
rat_tr = pd.read_csv('/gdrive/My Drive/Recom_Engine/train.txt', sep='\t', names=r_cols)
rat_tst = pd.read_csv('/gdrive/My Drive/Recom_Engine/test.txt', sep='\t', names=r_cols)
rat_tr.shape, rat_tst.shape

((90570, 4), (9430, 4))

In [0]:
rat_tr.head(), rat_tst.head()

(   user_id  movie_id  rating  unix_timestamp
 0        1         1       5       874965758
 1        1         2       3       876893171
 2        1         3       4       878542960
 3        1         4       3       876893119
 4        1         5       3       889751712,
    user_id  movie_id  rating  unix_timestamp
 0        1        20       4       887431883
 1        1        33       4       878542699
 2        1        61       4       878542420
 3        1       117       3       874965739
 4        1       155       2       878542201)

In [0]:
#calculating number of unique users and items to build a matrix
n_users = rating_df.user_id.unique().shape[0] # 943
n_items = rating_df.movie_id.unique().shape[0] # 1682)

In [0]:
#user item matrix to calculate similarity between users and item
data_matrix = np.zeros((n_users, n_items))
print(data_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
for line in rating_df.itertuples(): # to iterate over rows lines[1] means 2nd column of df
  data_matrix[line[1]-1, line[2]-1] = line[3]

# row = user , col = movie , value = rating

In [0]:
# calculating pairwise distance between user-user and item-item seperately

In [0]:
from sklearn.metrics.pairwise import pairwise_distances 

user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')


In [0]:
# prediction based on these similarity

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1) # mean along the row ,array of numbers
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) #  943 rows 1 column in mean_user_rating[] (column vector)
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [0]:
#predictions based on user-user similarity and item-item similarity.
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')


***Building a collaborative filtering model using Turicreate Library***

In [0]:
pip list


In [0]:
#train_data = turicreate.Sframe.read_csv('/gdrive/My Drive/Recom_Engine/train.txt', sep='\t', names=r_cols)
#test_data = turicreate.Sframe.read_csv('/gdrive/My Drive/Recom_Engine/train.txt', sep='\t', names=r_cols)
pip install -U turicreate==5.5.1

     |████████████████████████████████| 85.9MB 45kB/s 
     |████████████████████████████████| 23.8MB 69.3MB/s 
     |████████████████████████████████| 2.7MB 41.1MB/s 
     |████████████████████████████████| 327kB 42.3MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320850 sha256=6e4d09011da2a999f026830ec162fcfd1e52b138b4128c01ec43526de2a82cd9
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built resampy
ERROR: mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.17.5 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requirement requests<2.19.0,>=2.18.4, but you'll have requests 2.21.0 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: resampy 0.2.2
    Uninstalling resampy-0.2.2:
      Successfully uninstalled resampy-0.2.2


In [0]:
import turicreate

In [0]:
from turicreate import SFrame

In [0]:
train_data = turicreate.SFrame(rat_tr)
test_data = turicreate.SFrame(rat_tst)

In [0]:
'''will start with a simple popularity model and then build a collaborative filtering model.
build a model which will recommend movies based on the most popular choices, i.e., a model
 where all the users receive the same recommendation(s). We will use the turicreate recommender function popularity_recommender for this.'''

In [0]:
popularity_model = turicreate.popularity_recommender.create(train_data, user_id='user_id', item_id='movie_id', target='rating')


Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.112444s

90570 observations to process; with 1680 unique items.

In [0]:
# recommend the top 5 items for the first 5 users in our dataset.

popularity_recomm = popularity_model.recommend(users=[1,2,3,4,5],k=5)
popularity_recomm.print_rows(num_rows=25)

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1599   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    2    |   1599   |  5.0  |  1   |
|    2    |   1201   |  5.0  |  2   |
|    2    |   1189   |  5.0  |  3   |
|    2    |   1122   |  5.0  |  4   |
|    2    |   814    |  5.0  |  5   |
|    3    |   1599   |  5.0  |  1   |
|    3    |   1201   |  5.0  |  2   |
|    3    |   1189   |  5.0  |  3   |
|    3    |   1122   |  5.0  |  4   |
|    3    |   814    |  5.0  |  5   |
|    4    |   1599   |  5.0  |  1   |
|    4    |   1201   |  5.0  |  2   |
|    4    |   1189   |  5.0  |  3   |
|    4    |   1122   |  5.0  |  4   |
|    4    |   814    |  5.0  |  5   |
|    5    |   1599   |  5.0  |  1   |
|    5    |   1201   |  5.0  |  2   |
|    5    |   1189   |  5.0  |  3   |
|    5    | 

 collaborative filtering model. it's the item similarity model and make top 5 recommendations for the first 5 users.



In [0]:
#Training the model
item_sim_model = turicreate.item_similarity_recommender.create(train_data, user_id='user_id', item_id='movie_id', target='rating', similarity_type='cosine')

#Making recommendations
item_sim_recomm = item_sim_model.recommend(users=[1,2,3,4,5],k=5)
item_sim_recomm.print_rows(num_rows=25)

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.115511s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.27968s

+---------+----------+--------------------+------+
| user_id | movie_id |       score        | rank |
+---------+----------+--------------------+------+
|    1    |   423    | 0.987791502066241  |  1   |
|    1    |   202    | 0.9431346880115625 |  2   |
|    1    |   655    | 0.8001633629999088 |  3   |
|    1    |   568    | 0.7698910716380781 |  4   |
|    1    |   403    | 0.7614776537618564 |  5   |
|    2    |    50    | 1.1256258487701416 |  1   |
|    2    |   181    | 1.0272409457426805 |  2   |
|    2    |   121    | 0.9025371510248917 |  3   |
|    2    |    9     | 0.8518216059758112 |  4   |
|    2    |    7     | 0.8116787786667163 |  5   |
|    3    |   313    | 0.6353766620159149 |  1   |
|    3    |   328    | 0.6032880300825293 |  2   |
|    3    |   315    | 0.5422587123784152 |  3   |
|    3    |   331    | 0.5355071858926252 |  4   |
|    3    |   332    | 0.5316696112806146 |  5   |
|    4    |    50    | 1.1311477082116264 |  1   |
|    4    |   288    | 1.048715

In [0]:

model_performance = turicreate.recommender.util.compare_models(test_data, [popularity_model, item_sim_model])


PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    |          0.0           |          0.0           |
|   2    |          0.0           |          0.0           |
|   3    | 0.0003534817956875227  | 0.00010604453870625672 |
|   4    | 0.00026511134676564214 | 0.00010604453870625672 |
|   5    | 0.00021208907741251343 | 0.00010604453870625672 |
|   6    | 0.00017674089784376135 | 0.00010604453870625672 |
|   7    | 0.00015149219815179524 | 0.00010604453870625672 |
|   8    | 0.00013255567338282107 | 0.00010604453870625672 |
|   9    | 0.00011782726522917423 | 0.00010604453870625672 |
|   10   | 0.00021208907741251343 | 0.00021208907741251343 |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0417647969439832

Per User 